In [5]:
import time
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import torch

import optuna

sys.path.append("/jet/home/azhang19/stat 214/stat-214-lab2-group6/code/modeling")
from preprocessing import to_NCHW, pad_to_384x384, standardize_images
from classification import train_and_validate

device = "cuda" if torch.cuda.is_available() else "cpu"

torch.set_float32_matmul_precision('high')
torch.backends.cudnn.benchmark = True

use_amp = True

In [6]:
# Load and preprocess data
data = np.load("/jet/home/azhang19/stat 214/stat-214-lab2-group6/data/array_data.npz")
unlabeled_images, unlabeled_masks, labeled_images, labeled_masks, labels = data["unlabeled_images"], data["unlabeled_masks"], data["labeled_images"], data["labeled_masks"], data["labels"]

unlabeled_images = pad_to_384x384(to_NCHW(unlabeled_images))
unlabeled_masks = pad_to_384x384(unlabeled_masks)

labeled_images = pad_to_384x384(to_NCHW(labeled_images))
labeled_masks = pad_to_384x384(labeled_masks)
labels = pad_to_384x384(labels)

# Convert to tensors and move to GPU
unlabeled_images = torch.tensor(unlabeled_images, dtype=torch.float32).to(device)  # [161, 8, 384, 384]
unlabeled_masks = torch.tensor(unlabeled_masks, dtype=torch.bool).to(device)    # [161, 384, 384]

labeled_images = torch.tensor(labeled_images, dtype=torch.float32).to(device)      # [3, 8, 384, 384]
labeled_masks = torch.tensor(labeled_masks, dtype=torch.bool).to(device)        # [3, 384, 384]
labels = torch.tensor(labels, dtype=torch.long).to(device)                      # [3, 384, 384]


# Standardize images
unlabeled_images, std_channel, mean_channel = standardize_images(unlabeled_images, unlabeled_masks)
labeled_images, _, _ = standardize_images(labeled_images, labeled_masks, std_channel, mean_channel)

In [7]:
import pickle
with open("/jet/home/azhang19/optuna_study_original.pkl", "rb") as f:
    study = pickle.load(f)

In [13]:
feature = labeled_images
feature, _, _ = standardize_images(feature, labeled_masks)

record = np.zeros((10, 2))
classifiers = []

for repeat in range(10):

    train_val_idx = [0, 1]

    # Container for metrics from each fold
    fold_records_f1 = torch.zeros(len(train_val_idx))  # For F1 (objective)
    fold_records_acc = torch.zeros(len(train_val_idx))  # For accuracy (logging)

    classifiers_in_this_repeat = []
    # Assuming feature and labels are defined globally (e.g., torch tensors)
    for i in train_val_idx:
        # Leave-one-out style split
        train_idx = [j for j in train_val_idx if j != i]
        val_idx = [i]

        # Get training and validation data
        train_data = feature[train_idx]
        train_labels = labels[train_idx]
        val_data = feature[val_idx]
        val_labels = labels[val_idx]

        # Train and validate, get both F1 and accuracy
        classifier, val_f1, val_acc = train_and_validate(
            train_data=train_data,
            train_labels=train_labels,
            val_data=val_data,
            val_labels=val_labels,
            in_channels=feature.shape[1],
            num_layers=study.best_params["num_layers"],
            kernel_size=study.best_params["kernel_size"],
            hidden_channels=study.best_params["hidden_channels"],
            epochs=study.best_params["epochs"],
            lr=study.best_params["lr"],
            weight_decay=study.best_params["weight_decay"],
            optimizer_class=torch.optim.AdamW,
            loss_mix_ratio=study.best_params["loss_mix_ratio"],
            l1=study.best_params["l1"],
            class_weight="balanced",
            device=device,
            return_classifier=True,
        )
        classifiers_in_this_repeat.append((train_idx, classifier))

        fold_records_f1[i] = val_f1
        fold_records_acc[i] = val_acc

    classifiers.append(classifiers_in_this_repeat)
    avg_f1 = fold_records_f1.mean().item()
    avg_acc = fold_records_acc.mean().item()

    record[repeat] = [avg_f1, avg_acc]

record.mean(axis=0), record.std(axis=0)

(array([0.79999533, 0.84950012]), array([0.02407335, 0.01626197]))

In [18]:
# Repeat 0, trained on 1, and validated on 0
classifiers[0][0]

([1],
 ConvHead(
   (layers): Sequential(
     (0): ConvBlock(
       (conv): Conv2d(8, 54, kernel_size=(1, 1), stride=(1, 1), padding=same)
       (bn): BatchNorm2d(54, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (activation): SiLU()
     )
     (1): ConvBlock(
       (conv): Conv2d(54, 54, kernel_size=(1, 1), stride=(1, 1), padding=same)
       (bn): BatchNorm2d(54, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (activation): SiLU()
     )
     (2): Conv2d(54, 1, kernel_size=(1, 1), stride=(1, 1), padding=same)
   )
 ))

In [19]:
# validated on 0
classifiers[0][0][1](labeled_images[0:1])

tensor([[[[ 4.0932,  4.0932,  4.0932,  ...,  1.5397,  4.0932,  4.0932],
          [ 4.0932,  4.0932,  4.0932,  ...,  0.6036,  4.0932,  4.0932],
          [ 4.0932,  4.0932,  4.0932,  ..., -2.3120,  4.0932,  4.0932],
          ...,
          [ 4.0932,  4.0932,  4.0932,  ...,  4.0932,  4.0932,  4.0932],
          [ 4.0932,  4.0932,  4.0932,  ...,  4.0932,  4.0932,  4.0932],
          [ 4.0932,  4.0932,  4.0932,  ...,  4.0932,  4.0932,  4.0932]]]],
       device='cuda:0', grad_fn=<ConvolutionBackward0>)